# Influence of Events on President Approval Rate
By Paul P.<br>
This project was created by a collection of Trump Approval polls by [FiveThirtyEight](https://projects.fivethirtyeight.com/trump-approval-ratings/) and data scraped from the [Miller Center](https://millercenter.org/president/trump/key-events).

## Goals
The main objective of this project is to <b>analyze the change in approval rates</b> of a United States President <strong>in correlation to key events</strong> during their term. For this anaylsis, I will use poll data <em>(approval.csv)</em> of former President Trump. <br> 

We will organize this project with these questions:
1. What is the change of approval rate over time?
2. How do the events influence the approval rate?
3. What are the top influential events during the Trump Presidency?

## Retrieving Data (Web Scapring)

The poll data <em>(approval.csv)</em> we are using has been adjusted to address sample size uncertainties and house effects. I could not find a dataset for key events during the Trump Presidency so I will have to web scrap from the [Miller Center - Key Events](https://millercenter.org/president/trump/key-events) website.

In [1]:
# importing HTML parser
#import requests
#from bs4 import BeautifulSoup

Web scraping the data from each key event.

In [2]:
# website
URL = "https://millercenter.org/president/trump/key-events"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
for i in soup.findAll('time'):
        if i.has_attr('datetime'):
            print(i['datetime'])


NameError: name 'requests' is not defined

Web scarping the title of each key event. I won't use the description for this analysis. 

In [ ]:
mydivs = soup.find_all("span", {"class": "field-content"})
for element in mydivs:
    print(element.prettify())

<span class="field-content">
 Trump inaugurated
</span>
<span class="field-content">
 Women's March
</span>
<span class="field-content">
 Global gag rule reinstated
</span>
<span class="field-content">
 Withdrawal from the Trans-Pacific Partnership
</span>
<span class="field-content">
 Travel ban
</span>
<span class="field-content">
 Gorsuch nominated for SCOTUS
</span>
<span class="field-content">
 Flynn resigns
</span>
<span class="field-content">
 New executive order on the travel ban
</span>
<span class="field-content">
 Reversal on environmental protection
</span>
<span class="field-content">
 Action against Syrian government
</span>
<span class="field-content">
 Neil M. Gorsuch joins the US Supreme Court
</span>
<span class="field-content">
 FBI director James Comey removed
</span>
<span class="field-content">
 Robert Muller chosen as special counsel
</span>
<span class="field-content">
 Withdrawal from Paris climate accord
</span>
<span class="field-content">
 Chief of Staff Kel

Now that we have the key events, I will bring them to Excel to do a quick data cleaning and save them to a csv. <em>(Events.csv)</em> 

And now that we have our data, we can continue to importing to Pandas and doing the data manipulation needed for our analysis.

## Reading and Manipulating Data

First, I will read the two seperate `.csv` files into a **Pandas** dataframe.

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

NameError: name 'sns' is not defined

In [ ]:
df = pd.read_csv('approval.csv')
df['enddate'] = df['enddate'].astype('datetime64[ns]')
df.head()

In [ ]:
# reading the Events spreadsheet that we web scraped earlier
df_ev = pd.read_csv('Events.csv')
df_ev.Date = df_ev.Date.astype('datetime64[ns]')

# renaming our date column for df merge purposes
df_ev['enddate'] = df_ev['Date']
df_ev = df_ev.set_index('enddate').drop('Date', axis=1)

df_ev.head()

Since there are multiple polls per day, we will find the mean for each day and set that as the approval rate for that day. Since we want to make a lineplot with the polls, it's also important to summarize the data for a cleaner line. We will use `rolling()` and `mean()` for this.

In [ ]:

df_rate = df[['adjusted_approve', 'enddate']].set_index('enddate')
df_rate = df_rate.rolling(min_periods=1, window=100).mean().dropna()


df_rate = df_rate.groupby(pd.Grouper(freq='D')).mean()

df_rate = df_rate.rename(columns={'adjusted_approve': 'Approval_Rate'})
df_rate.head()

Then we will merge the two dataframes so we know what the approval rate was at the **key event** date.

In [ ]:
# merge key events df with approval rates df
df_ev_rate = pd.merge(df_rate, df_ev, left_index=True, right_index=True).reset_index()

df_ev_rate.head()

## Analyzing the Data

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(11,5))

ax = sns.kdeplot(data=df, x='enddate', y='adjusted_approve', levels=100, thresh=0, fill=True, cmap='mako', ax=axes[0])
ax1 = sns.kdeplot(data=df, x='enddate', y='adjusted_disapprove', color='w', levels=100, thresh=0, fill=True, cmap='rocket', ax1=axes[1])

ax.set_ylim(33,61)
ax1.set_ylim(33,61)
sns.set(rc={'axes.facecolor': 'black', 'grid.color': 'black'})

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,5))
sns.kdeplot(data=df, x='adjusted_approve', hue='subgroup', fill=True, alpha=.3, ax=axes[0], palette='mako')
sns.kdeplot(data=df, x='adjusted_disapprove', hue='subgroup', fill=True, alpha=.3, ax=axes[1], palette='flare')
sns.set_style('darkgrid')

In [ ]:
Capitol = df_ev_rate.loc[df_ev_rate['enddate'] == '2021-01-06']
Impeach = df_ev_rate.loc[df_ev_rate['enddate'] == '2019-12-18']
Drone = df_ev_rate.loc[df_ev_rate['enddate'] == '2020-03-27']
Tax_Bill = df_ev_rate.loc[df_ev_rate['enddate'] == '2017-12-22']
FBI = df_ev_rate.loc[df_ev_rate['enddate'] == '2017-05-09']

In [ ]:
palette = {'Approval_Rate': 'dodgerblue'}
plt.figure(figsize=(12,8))
sns.set_style('darkgrid')
Labels = ['']

sns.lineplot(data=df_rate, palette=palette, zorder=5).set(xlabel='Date')
sns.scatterplot(data=df_ev_rate, x='enddate', y='Approval_Rate', c=['midnightblue'], s=15, zorder=10, label='Key Event')
plt.title('Trump Approval Rate')

plt.annotate(xy=(Capitol.enddate, Capitol.Approval_Rate), xytext=(Capitol.enddate+ pd.Timedelta(weeks=-40), 39), text='U.S Capitol Mob Attack', arrowprops=dict(color='black', arrowstyle='-'))

plt.annotate(xy=(Impeach.enddate, Impeach.Approval_Rate), xytext=(Impeach.enddate+ pd.Timedelta(weeks=-30), 44), text='Trump impeached', arrowprops=dict(color='black', arrowstyle='-'))

plt.annotate(xy=(Drone.enddate, Drone.Approval_Rate), xytext=(Drone.enddate+ pd.Timedelta(weeks=-48), 45.5), text='COVID Relief Efforts (CARES)', arrowprops=dict(color='black', arrowstyle='-'))

plt.annotate(xy=(FBI.enddate, FBI.Approval_Rate), xytext=(FBI.enddate+ pd.Timedelta(weeks=+14), 44), text='Trump fires FBI Director', arrowprops=dict(color='black', arrowstyle='-'))

plt.annotate(xy=(Tax_Bill.enddate, Tax_Bill.Approval_Rate), xytext=(Tax_Bill.enddate+ pd.Timedelta(weeks=+24), 39), text='Tax bill signed', arrowprops=dict(color='black', arrowstyle='-'))

plt.annotate(xy=(['2017-01-20'], 36.7), xytext=(['2017-02-20'], 36.7), text='Trump Inauguration', arrowprops=dict(color='black', arrowstyle='-'))

plt.axvline(x=['2017-01-20'], linestyle='--', color='black', alpha=0.3)


In [ ]:
df_pct_change = df_rate

df_pct_change = df_pct_change.pct_change().dropna()
df_pct_change = pd.merge(df_pct_change, df_ev, left_index=True, right_index=True).reset_index().dropna()


df_pct_change.head()

In [ ]:
df_pct_change_top = df_pct_change.sort_values(by='Approval_Rate', key=abs, ascending=False, ignore_index=True)
df_pct_change_top['Absolute_Change'] = df_pct_change_top['Approval_Rate'].abs()
df_pct_change_top = df_pct_change_top[:10]
df_pct_change_top